# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sqlalchemy import create_engine
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
nltk.download(['punkt','stopwords', 'wordnet', 'averaged_perceptron_tagger'])
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
# load data from database
engine = create_engine('sqlite:///MessageDatabase.db')
df = pd.read_sql_table('MessageDatabase', engine)  
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)


### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [10]:
#THIS IS THE PIPELINE DESIGN IMPLEMENTED
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def display_results(y_test, y_pred):
    y_pred_names = pd.DataFrame(y_pred, columns = y_test.columns)
    for column in y_test.columns:
        print('column_name: {}\n'.format(column))
        print(classification_report(y_test[column], y_pred_names[column]))

In [14]:
y_pred = pipeline.predict(X_test)
display_results(y_test, y_pred)

column_name: related

             precision    recall  f1-score   support

          0       0.55      0.43      0.48      1542
          1       0.83      0.89      0.86      4966
          2       0.57      0.35      0.43        37

avg / total       0.76      0.78      0.77      6545

column_name: request

             precision    recall  f1-score   support

          0       0.88      0.99      0.93      5417
          1       0.83      0.35      0.49      1128

avg / total       0.87      0.88      0.85      6545

column_name: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6513
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6545

column_name: aid_related

             precision    recall  f1-score   support

          0       0.68      0.91      0.78      3826
          1       0.76      0.40      0.52      2719

avg / total       0.71      0.70      0.67      654

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
from sklearn.neighbors import KNeighborsClassifier
rfc_classifier = RandomForestClassifier()
multi_rfc_classifier = MultiOutputClassifier(rfc_classifier)

pipeline_2 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf2', multi_rfc_classifier)
])



# Define hyperparameters to tune
param_grid2 = {

    'clf2__estimator__min_samples_leaf': (1,2),
    'clf2__estimator__min_samples_split':(2,3,4)
}
#pipeline_2.get_params()

In [45]:
clf2 = GridSearchCV(pipeline_2, param_grid=param_grid2, verbose=2, n_jobs=-1)
clf2.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4 
[CV] clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4 
[CV] clf2__estimator__min_samples_leaf=2, clf2__estimator__min_samples_split=2 
[CV] clf2__estimator__min_samples_leaf=2, clf2__estimator__min_samples_split=2 
[CV] clf2__estimator__min_samples_leaf=2, clf2__estimator__m

[Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed: 43.6min remaining: 68.6min


[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4, total=32.5min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4, total=32.5min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3, total=32.9min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=4, total=32.7min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3, total=33.4min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=3, total=33.7min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2, total=34.0min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2, total=33.9min
[CV]  clf2__estimator__min_samples_leaf=1, clf2__estimator__min_samples_split=2, total=34.4min
[CV]  clf2__estimator__min_samples_leaf=2, clf2__estimator__min_samples_split=4, total= 5.2min
[CV]  clf2__estimator__min_samples_leaf=2, clf2__e

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 49.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf2__estimator__min_samples_leaf': (1, 2), 'clf2__estimator__min_samples_split': (2, 3, 4)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

In [46]:
#pipeline_2.get_params().keys()

In [47]:
# Print the best hyperparameters
print("Best Hyperparameters:", clf2.best_params_)

# Evaluate the model on the test set
accuracy = clf2.score(X_test, y_test)
print("Test Accuracy:", accuracy)

Best Hyperparameters: {'clf2__estimator__min_samples_leaf': 1, 'clf2__estimator__min_samples_split': 2}
Test Accuracy: 0.225668449198


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [54]:
y_pred = clf2.predict(X_test)

In [55]:
def get_metrics(test_value, predicted_value):
    """
    get_metrics calculates f1 score, accuracy and recall

    Args:
        test_value (list): list of actual values
        predicted_value (list): list of predicted values

    Returns:
        dictionray: a dictionary with accuracy, f1 score, precision and recall
    """
    accuracy = accuracy_score(test_value,predicted_value)
    precision =round( precision_score(test_value,predicted_value,average='micro'))
    recall = recall_score(test_value,predicted_value,average='micro')
    f1 = f1_score(test_value,predicted_value,average='micro')
    return {'Accuracy':accuracy, 'f1 score':f1,'Precision':precision, 'Recall':recall}

In [58]:
from sklearn.metrics import  f1_score,precision_score,recall_score,accuracy_score,make_scorer
train_results = []
for i,column in enumerate(y_train.columns):
    result = get_metrics(y_test.loc[:,column].values,y_pred[:,i])
    train_results.append(result)
train_results_df = pd.DataFrame(train_results)
train_results_df

,Accuracy,Precision,Recall,f1 score
0,0.794805,1.0,0.794805,0.794805
1,0.874408,1.0,0.874408,0.874408
2,0.993736,1.0,0.993736,0.993736
3,0.733537,1.0,0.733537,0.733537
4,0.919633,1.0,0.919633,0.919633
5,0.954163,1.0,0.954163,0.954163
6,0.969137,1.0,0.969137,0.969137
7,0.978762,1.0,0.978762,0.978762
8,0.970512,1.0,0.970512,0.970512
9,1.000000,1.0,1.000000,1.000000


In [ ]:
#burayı henüz test etmedim.
#accuracy = (y_pred == y_test).mean()
#print("Accuracy:", accuracy)
##print("\nBest Parameters:", cv.best_params_)
#display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [48]:
#LogisticRegression kullanabilirsin
#Word2Vec TFIDF yerine kullanılabilir.
rfc_classifier = RandomForestClassifier()
multi_rfc_classifier = MultiOutputClassifier(rfc_classifier)

pipeline_3 = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor()),
        #('word2vec', Word2VecExtractor(model_path='word2vec_model.bin')),
        ('num_chars', NumCharsExtractor())
    ])),

    ('clf3', multi_rfc_classifier)
])



# Define hyperparameters to tune
param_grid2 = {

    'clf3__estimator__min_samples_leaf': (1,2),
    'clf3__estimator__min_samples_split':(2,3,4)
}
#pipeline_2.get_params()

NameError: name 'Word2VecExtractor' is not defined

### 9. Export your model as a pickle file

In [10]:
import pickle
pickle.dump(pipeline, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.